# Hands On "Observing the Earth from your Jupyter Notebook"

Dans cette démo, nous allons manipuler les concepts "haut niveau" vu précédemment: 
- STAC
- XARRAY

En utilisant https://stackstac.readthedocs.io

Le but de cette démo est de rester "haut niveau" pour voir "l'état final des choses". 

Dans le prochain cours nous allons construire nous même tous les éléments un par un

In [ ]:
# !pip install dask[distributed] stackstac pystac_client rasterio geopandas shapely geojson geojsonio

In [ ]:
import json
import geojsonio
import dask.utils
import geogif
import geojson
import geopandas
import pystac_client
import shapely.geometry
import stackstac
from rasterio.enums import Resampling

Tout d'abord nous allons nous connecter au catalog des images Sentinel 2 sur AWS 

https://registry.opendata.aws/sentinel-2-l2a-cogs/

In [ ]:
STAC_URL = "https://earth-search.aws.element84.com/v0"

Nous allons rechercher toutes les images centrées sur Toulouse (ou toute autre localisation que vous souhaitez).

Allez sur [geojson.io](http://geojson.io/#map=13/43.6043/1.4068) et dessinez une zone d'intérêt autour de Toulouse

Exemple: 

```python
aoi = {
    "type": "Polygon",
    "coordinates": [
        [
            [1.3260841369628906, 43.54568641528668],
            [1.4970588684082031, 43.54568641528668],
            [1.4970588684082031, 43.651106046724365],
            [1.3260841369628906, 43.651106046724365],
            [1.3260841369628906, 43.54568641528668],
        ]
    ],
}
```

Ensuite, on sélectionne une plage temporelle

In [ ]:
aoi = ...
date_min, date_max = "2021-01-01", "2021-12-31"

Lançons une requête sur le catalogue

In [ ]:
items = (
    pystac_client.Client.open(STAC_URL)
    .search(
        intersects=aoi,
        collections=["sentinel-s2-l2a-cogs"],
        datetime="{}/{}".format(date_min, date_max),
        limit=10_000,
    )
    .get_all_items()
)
len(items)

Regardons un peu les "Items"

In [ ]:
from copy import deepcopy

import geopandas as gpd
import pandas as pd
from shapely.geometry import shape


# convert a list of STAC Items into a GeoDataFrame
def items_to_geodataframe(items):
    _items = []
    for i in items:
        _i = deepcopy(i)
        _i["geometry"] = shape(_i["geometry"])
        _items.append(_i)
    gdf = gpd.GeoDataFrame(pd.json_normalize(_items))
    for field in ["properties.datetime", "properties.created", "properties.updated"]:
        if field in gdf:
            gdf[field] = pd.to_datetime(gdf[field])
    gdf.set_index("properties.datetime", inplace=True)
    return gdf


# convert found items to a GeoDataFrame
items_gdf = items_to_geodataframe(items.to_dict()["features"])
items_gdf.head()

In [ ]:
features = []

for date, row in items_gdf.iterrows():
    features.append(geojson.Feature(geometry=shapely.geometry.mapping(row.get("geometry"))))

geojsonio.display(features)

On charge les items dans stackstac pour créer notre série temporelle.

Par défaut, le tableau créé est "lazy"

In [ ]:
RESOLUTION = 50  # Resample to 50m to avoid heavy computations

In [ ]:
BOUNDS = shapely.geometry.shape(aoi).bounds

In [ ]:
stack = stackstac.stack(
    items,
    resolution=RESOLUTION,
    bounds_latlon=BOUNDS,
    resampling=Resampling.bilinear,
    assets=["B04", "B03", "B02"],
)

In [ ]:
stack

In [ ]:
type(stack.data)

Select the visibles bands in RGB order

In [ ]:
rgb = stack.sel(band=["B04", "B03", "B02"])
rgb

Faisons une série temporelle "composite" en utilisant la médiane

In [ ]:
monthly_rgb = rgb.resample(time="MS").median(dim="time")
monthly_rgb

Maintenant, nous allons lancer le calcul de notre série temporelle. Notez que jusqu'alors l'array était "lazy" c'est à dire qu'on ne manipulait que des instructions.

Maintenant nous allons effectuer le calcul : 

In [ ]:
rgb = monthly_rgb.compute()

In [ ]:
rgb

On note que le tableau est maintenant en format numpy

In [ ]:
type(rgb.data)

On peut maintenant afficher les données

In [ ]:
rgb.plot.imshow(row="time", rgb="band", robust=True, size=6);

Transformons ceci dans un gif !

In [ ]:
gif = geogif.gif(rgb, fps=1, vmin=0, vmax=2048)

In [ ]:
gif

## Et voilà !

Vous pouvez aller plus loin avec les tutoriaux suivants:

https://stackstac.readthedocs.io/en/latest/examples/index.html

https://stackstac.readthedocs.io/en/latest/examples/gif.html

https://stackstac.readthedocs.io/en/latest/basic.html

Dans les prochains BE, nous allons voir les détails qui se cachent derrière tout ça !

## Pour aller plus loin

- https://examples.dask.org/array.html
- https://examples.dask.org/applications/satellite-imagery-geotiff.html
- https://xarray-contrib.github.io/xarray-tutorial/scipy-tutorial/00_overview.html